In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
import random

from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
datadirectory = "archive/train/"

In [3]:
Classes = ["angry","disgust","fear","happy","neutral","sad","surprise"]


In [4]:
for category in Classes:
    path = os.path.join(datadirectory , category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
       
        break
    break
        
   
        

In [5]:
img_size = 224  #Image net 224*224 for transfer learning
new_array = cv2.resize(img_array,(img_size,img_size))




In [6]:
#reading all the images and converting them to array

In [7]:
training_Data =[] #data array

def create_training_Data():
    for category in Classes:
        path = os.path.join(datadirectory,category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(img_size,img_size))
                training_Data.append([new_array,class_num]) 
            except Exception as e:
                pass

In [8]:
create_training_Data()

In [9]:
print(len(training_Data))

28709


In [10]:

random.shuffle(training_Data)

In [11]:
X = []   ##data/feature  ##resized array
Y = []  ##label -  index created - class label  

for features,label in training_Data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1,img_size,img_size,3) ##numpy array is created of four dimension

In [12]:
Y[0]

4

In [13]:
type(Y)

list

In [14]:
Y = np.array(Y)

In [15]:
Y.shape

(28709,)

In [16]:
##Classes = ["angry","disgust","fear","happy","neutral","sad","surprise"]
## list in python have index [0,1,2,..]
# so these are treated as indexes and along with resized array are stored together for each image
## 3 refers to rgb

In [17]:
X.shape

(28709, 224, 224, 3)

In [18]:
model = tf.keras.applications.MobileNetV2()

In [19]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              


#transfer learning - tuning weights will start from last check point

In [20]:
base_input = model.layers[0].input

In [21]:
base_output = model.layers[-2].output

In [22]:
base_output

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [23]:
final_output = layers.Dense(128)(base_output)  ##adding new layer after the output of the global pooling layer
final_output = layers.Activation('relu')(final_output)  ##activation layer
final_output = layers.Dense(64)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output) ## 07 classes


In [24]:
final_output

<KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'dense_2')>

In [2]:
new_model = keras.Model(inputs = base_input , outputs = final_output)

NameError: name 'base_input' is not defined

In [26]:
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [27]:
new_model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [28]:
new_model.fit(X,Y,epochs = 25)

Epoch 1/25
898/898 [==============================] - 1464s 2s/step - loss: 1.2254 - accuracy: 0.5403
Epoch 2/25
898/898 [==============================] - 1524s 2s/step - loss: 1.0483 - accuracy: 0.6078
Epoch 3/25
898/898 [==============================] - 1378s 2s/step - loss: 0.9685 - accuracy: 0.6395
Epoch 4/25
898/898 [==============================] - 1705s 2s/step - loss: 0.9224 - accuracy: 0.6558
Epoch 5/25
898/898 [==============================] - 1579s 2s/step - loss: 0.8619 - accuracy: 0.6811
Epoch 6/25
898/898 [==============================] - 2406s 3s/step - loss: 0.8186 - accuracy: 0.6963
Epoch 7/25
898/898 [==============================] - 1375s 2s/step - loss: 0.7621 - accuracy: 0.7220
Epoch 8/25
898/898 [==============================] - 1367s 2s/step - loss: 0.7073 - accuracy: 0.7376
Epoch 9/25
898/898 [==============================] - 1483s 2s/step - loss: 0.6564 - accuracy: 0.7585
Epoch 10/25
898/898 [==============================] - 1430s 2s/step - loss: 0.593

In [29]:
new_model.save('Final_model_95p07.h5')

In [2]:
new_model = tf.keras.models.load_model('Final_model_95p07.h5')

In [5]:
frame = cv2.imread("happy.jpg")

In [6]:
img = load_img(frame, target_size=(48, 48))

TypeError: path should be path-like or io.BytesIO, not <class 'numpy.ndarray'>

In [7]:
frame.shape

(428, 600, 3)

In [ ]:
img_size = 224  #Image net 224*224 for transfer learning
new_array = cv2.resize(img_array,(img_size,img_size))

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [7]:
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    """
    Load an image from a file, resize it to the target size, and return it as a NumPy array.
    
    Parameters:
    img_path (str): The file path to the image.
    target_size (tuple): The desired size of the output image, default is (224, 224).
    
    Returns:
    np.ndarray: The preprocessed image as a NumPy array.
    """
    # Read the image from file
    img = cv2.imread(img_path)
    
    # Check if the image was successfully loaded
    if img is None:
        raise ValueError(f"Failed to load image at {img_path}")
    
    # Resize the image to the target size
    resized_img = cv2.resize(img, target_size)
    
    # Convert the image to float32 and normalize to the range [0, 1]
    normalized_img = resized_img.astype(np.float32) / 255.0
    
    return normalized_img

# Example usage
img_path = 'suraj.jpg'
image_array = load_and_preprocess_image(img_path)

print(f"Image array shape: {image_array.shape}")  # Should be (224, 224, 3)

Image array shape: (224, 224, 3)


In [ ]:
plt.imshow(image_array)

In [ ]:


def load_and_preprocess_image(img_path, target_size=(224, 224)):
    """
    Load an image from a file, resize it to the target size, and return it as a NumPy array.
    
    Parameters:
    img_path (str): The file path to the image.
    target_size (tuple): The desired size of the output image, default is (224, 224).
    
    Returns:
    np.ndarray: The preprocessed image as a NumPy array.
    """
    # Read the image from file
    img = cv2.imread(img_path)
    
    # Check if the image was successfully loaded
    if img is None:
        raise ValueError(f"Failed to load image at {img_path}")
    
    # Resize the image to the target size
    resized_img = cv2.resize(img, target_size)
    
    # Convert the image to float32 and normalize to the range [0, 1]
    normalized_img = resized_img.astype(np.float32) / 255.0
    
    return normalized_img

def show_image(img_array):
    """
    Display an image using Matplotlib.
    
    Parameters:
    img_array (np.ndarray): The image to display.
    """
    plt.imshow(img_array)
    plt.axis('off')  # Hide the axis
    plt.show()

# Example usage
img_path = 'happy.jpg'
image_array = load_and_preprocess_image(img_path)

# Show the preprocessed image
show_image(image_array)
